# Import

In [3]:
# ================================================================
# 📦 IMPORTS GÉNÉRAUX
# ================================================================
import os, gc
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from google.colab import files  # pour le téléchargement auto
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
import golois  # Librairie C++ fournie (jeu de données Go)
from tensorflow.keras import layers, regularizers
from tensorflow import keras
import gc

import json
import matplotlib.pyplot as plt
from google.colab import files



# Utilitaires internes (si tu veux factoriser ton code)
# from utils.model_blocks import dw_conv_block, se_block, attention_residual_block
# from utils.training_utils import moving_average, save_plots


ImportError: dlopen(/Users/adamhannachi/DeepLearning/DeDeepLearning_GO/DeepLearning_GO/golois.so, 0x0002): tried: '/Users/adamhannachi/DeepLearning/DeDeepLearning_GO/DeepLearning_GO/golois.so' (slice is not valid mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/adamhannachi/DeepLearning/DeDeepLearning_GO/DeepLearning_GO/golois.so' (no such file), '/Users/adamhannachi/DeepLearning/DeDeepLearning_GO/DeepLearning_GO/golois.so' (slice is not valid mach-o file)

# Modèle

## Paramètres

In [ ]:
# ================================================================
# ⚙️ PARAMÈTRES GÉNÉRAUX + NOMENCLATURE STANDARDISÉE
# ================================================================

import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime

# -------------------------------
# 🔹 Informations générales
# -------------------------------
trainer = "AdamH"                      # 🔸 ton identifiant / prénom (modifiable)
base_title = "Model4B_AttentionHybrid" # 🔸 nom du modèle / projet
date_str = datetime.now().strftime("%Y%m%d")  # 📅 date format YYYYMMDD

# -------------------------------
# 🔹 Hyperparamètres principaux
# -------------------------------
planes = 31
moves = 361
N = 10000
epochs = 500
batch = 128
filters = 110
se_reduction = 10
value_dense_units = 56
l2_reg = 1e-4
learning_rate = 1e-4

# -------------------------------
# 🔹 Nomenclature du modèle (automatique)
# -------------------------------
model_name = f"{date_str}_{trainer}_{base_title}_ep{epochs}_bs{batch}_lr{learning_rate}_l2reg{l2_reg}"
print(f"🧾 Nom du modèle généré : {model_name}")

# -------------------------------
# 🔹 Dossier de sortie
# -------------------------------
output_root = "output"
output_dir = os.path.join(output_root, model_name)

# Supprime le dossier s'il existe déjà (run propre)
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

print(f"📂 Dossier de sortie créé : {output_dir}")

# -------------------------------
# 🔹 Allocation des tenseurs factices (à remplacer par vraies données)
# -------------------------------
input_data = np.zeros((N, 19, 19, planes), dtype=np.float32)
policy      = np.zeros((N, moves), dtype=np.float32)
value       = np.zeros((N,), dtype=np.float32)
end         = np.zeros((N, 19, 19, 2), dtype=np.float32)
groups      = np.zeros((N, 19, 19, 1), dtype=np.float32)



## Définition du modèle

In [ ]:
# ================================================================
# 🧠 DÉFINITION DU MODÈLE - ATTENTION HYBRID (<100k params)
# ================================================================


# ---------------------
# 🔹 BLOCS AVANCÉS
# ---------------------
def dw_conv_block(x, filters, l2_reg, stride=1):
    """Depthwise separable convolution (MobileNet style)"""
    x = layers.SeparableConv2D(
        filters, 3, strides=stride, padding='same', activation='relu',
        depthwise_regularizer=regularizers.l2(l2_reg),
        pointwise_regularizer=regularizers.l2(l2_reg)
    )(x)
    x = layers.BatchNormalization()(x)
    return x

def se_block(x, reduction=8):
    """Squeeze & Excitation attention block"""
    c = int(x.shape[-1])
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Dense(c // reduction, activation='relu')(se)
    se = layers.Dense(c, activation='sigmoid')(se)
    se = layers.Reshape((1, 1, c))(se)
    return layers.Multiply()([x, se])

def attention_residual_block(x, filters, l2_reg, reduction):
    """Residual block combinant attention et convolutions séparables"""
    shortcut = x
    x = dw_conv_block(x, filters, l2_reg)
    x = dw_conv_block(x, filters, l2_reg)
    x = se_block(x, reduction=reduction)
    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)
    return x

# ---------------------
# 🔹 CONSTRUCTION DU MODÈLE
# ---------------------
inp = keras.Input(shape=(19, 19, planes), name='board')

x = layers.Conv2D(filters, 1, activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(l2_reg))(inp)
x = layers.BatchNormalization()(x)

for i in range(3):
    x = attention_residual_block(x, filters, l2_reg, reduction=se_reduction)
    if i % 2 == 1:
        x = layers.Dropout(0.10)(x)

# --- Policy Head ---
p = layers.Conv2D(1, 1, activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(l2_reg))(x)
p = layers.Flatten()(p)
policy_head = layers.Activation('softmax', name='policy')(p)

# --- Value Head ---
v = layers.GlobalAveragePooling2D()(x)
v = layers.Dense(value_dense_units, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg))(v)
v = layers.Dropout(0.15)(v)
value_head = layers.Dense(1, activation='sigmoid', name='value',
                          kernel_regularizer=regularizers.l2(l2_reg))(v)

# ---------------------
# 🔹 COMPILATION
# ---------------------
model = keras.Model(inputs=inp, outputs=[policy_head, value_head])

optimizer = keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0)
policy_loss = keras.losses.CategoricalCrossentropy(label_smoothing=0.05)
value_loss  = keras.losses.Huber(delta=0.1)
top5 = keras.metrics.TopKCategoricalAccuracy(k=5, name='top5')

model.compile(
    optimizer=optimizer,
    loss={'policy': policy_loss, 'value': value_loss},
    loss_weights={'policy': 1.0, 'value': 1.0},
    metrics={'policy': ['categorical_accuracy', top5], 'value': ['mae']}
)

total_params = model.count_params()
print(f"✅ {model_name} prêt : {total_params} paramètres totaux (<100k attendu)\n")
assert total_params < 100_000, f"❌ Trop de paramètres : {total_params} (limite <100k)"
model.summary()


## Train


In [ ]:
# ================================================================
# 🏋️‍♂️ ENTRAÎNEMENT DU MODÈLE
# ================================================================

# 🔹 Callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(output_dir, f"{model_name}_best.keras"),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.CSVLogger(
        os.path.join(output_dir, f"{model_name}_training_log.csv"),
        append=False
    )
]

# 🔹 Historique d'entraînement
history_all = {
    "loss": [], "policy_acc": [], "policy_top5": [], "value_mae": [],
    "val_loss": [], "val_policy_acc": [], "val_policy_top5": [], "val_value_mae": [], "val_epochs": []
}

# 🔹 Boucle d'entraînement principale
with open(os.path.join(output_dir, f"{model_name}_results.txt"), "w") as f:
    for epoch in range(1, epochs + 1):
        print(f"\n--- Epoch {epoch}/{epochs} ---")
        golois.getBatch(input_data, policy, value, end, groups, epoch * N)
        do_val = (epoch % 5 == 0) or (epoch == epochs)
        validation_data = None
        if do_val:
            golois.getValidation(input_data, policy, value, end)
            validation_data = (input_data, [policy, value])

        hist = model.fit(
            input_data, [policy, value],
            epochs=1, batch_size=batch, verbose=1,
            validation_data=validation_data,
            callbacks=callbacks if do_val else None
        )

        history_all["loss"].append(hist.history["loss"][0])
        history_all["policy_acc"].append(hist.history["policy_categorical_accuracy"][0])
        history_all["policy_top5"].append(hist.history["policy_top5"][0])
        history_all["value_mae"].append(hist.history["value_mae"][0])

        if do_val:
            history_all["val_epochs"].append(epoch)
            history_all["val_loss"].append(hist.history["val_loss"][0])
            history_all["val_policy_acc"].append(hist.history["val_policy_categorical_accuracy"][0])
            history_all["val_policy_top5"].append(hist.history["val_policy_top5"][0])
            history_all["val_value_mae"].append(hist.history["val_value_mae"][0])
            f.write(f"Epoch {epoch}: val_loss={history_all['val_loss'][-1]:.4f}, "
                    f"val_acc={history_all['val_policy_acc'][-1]:.4f}, "
                    f"val_top5={history_all['val_policy_top5'][-1]:.4f}, "
                    f"val_value_mae={history_all['val_value_mae'][-1]:.4f}\n")

        if epoch % 5 == 0:
            gc.collect()

print(f"\n✅ Entraînement terminé pour {model_name}")


# Sauvegarde

In [ ]:
# ================================================================
# 💾 SAUVEGARDE + 📊 VISUALISATION + 📈 LISSAGE (dans /output)
# ================================================================

import json
import matplotlib.pyplot as plt
from google.colab import files

# 🔹 Création du dossier (si jamais supprimé ou manquant)
os.makedirs(output_dir, exist_ok=True)

# 🔹 Sauvegarde du modèle (.h5 & .keras)
h5_path = os.path.join(output_dir, f"{model_name}.h5")
keras_path = os.path.join(output_dir, f"{model_name}.keras")
hist_json = os.path.join(output_dir, "history.json")

model.save(h5_path)
model.save(keras_path)
print(f"✅ Modèles sauvegardés :\n - {h5_path}\n - {keras_path}")

# 🔹 Sauvegarde de l’historique complet
with open(hist_json, "w") as f:
    json.dump(history_all, f)
print(f"📘 Historique sauvegardé : {hist_json}")

# 🔹 Téléchargement automatique du modèle sur Colab
files.download(h5_path)
print(f"📥 Téléchargement automatique du modèle {h5_path} terminé ✅")

# ================================================================
# 📊 VISUALISATION DES COURBES
# ================================================================
plt.style.use("seaborn-v0_8-whitegrid")

def save_plot(fig, name):
    """Sauvegarde une figure dans le dossier du modèle"""
    fig.savefig(os.path.join(output_dir, name), dpi=300, bbox_inches='tight')

epochs_range = range(1, len(history_all["loss"]) + 1)

# --- 1️⃣ Courbe de perte (Loss)
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(epochs_range, history_all["loss"], label='Training Loss', linewidth=2)
if history_all["val_loss"]:
    ax.plot(history_all["val_epochs"], history_all["val_loss"], label='Validation Loss', linewidth=2)
ax.set_title(f"{model_name} - Loss")
ax.set_xlabel("Epochs"); ax.set_ylabel("Loss")
ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
save_plot(fig, "loss_curve.png")
plt.close(fig)

# --- 2️⃣ Policy Accuracy
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(epochs_range, history_all["policy_acc"], label='Policy Accuracy', linewidth=2)
if history_all["val_policy_acc"]:
    ax.plot(history_all["val_epochs"], history_all["val_policy_acc"], '--', label='Val Policy Acc', linewidth=2)
ax.set_title(f"{model_name} - Policy Accuracy")
ax.set_xlabel("Epochs"); ax.set_ylabel("Accuracy")
ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
save_plot(fig, "policy_accuracy.png")
plt.close(fig)

# --- 3️⃣ Top-5 Accuracy
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(epochs_range, history_all["policy_top5"], label='Top-5 Accuracy', linewidth=2)
if history_all["val_policy_top5"]:
    ax.plot(history_all["val_epochs"], history_all["val_policy_top5"], '--', label='Val Top-5', linewidth=2)
ax.set_title(f"{model_name} - Top-5 Accuracy")
ax.set_xlabel("Epochs"); ax.set_ylabel("Accuracy")
ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
save_plot(fig, "top5_accuracy.png")
plt.close(fig)

# --- 4️⃣ Value MAE
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(epochs_range, history_all["value_mae"], label='Value MAE', linewidth=2)
if history_all["val_value_mae"]:
    ax.plot(history_all["val_epochs"], history_all["val_value_mae"], '--', label='Val Value MAE', linewidth=2)
ax.set_title(f"{model_name} - Value MAE")
ax.set_xlabel("Epochs"); ax.set_ylabel("MAE")
ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
save_plot(fig, "value_mae.png")
plt.close(fig)

# ================================================================
# 📈 LISSAGE DE LA POLICY ACCURACY
# ================================================================
def moving_average(data, window=5):
    data = np.array(data)
    if len(data) < window:
        return data
    return np.convolve(data, np.ones(window)/window, mode='valid')

if history_all["policy_acc"]:
    smooth_acc = moving_average(history_all["policy_acc"], 5)
    epochs_smooth = np.arange(5, len(history_all["policy_acc"]) + 1)
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(range(1, len(history_all["policy_acc"]) + 1),
            history_all["policy_acc"], alpha=0.4, label='Brut', linewidth=1.5)
    ax.plot(epochs_smooth, smooth_acc, color='red', linewidth=2.5,
            label='Moyenne glissante (5)')
    ax.set_title(f"{model_name} - Policy Accuracy (Lissée)")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Accuracy")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.8)
    save_plot(fig, "policy_acc_smooth.png")
    plt.close(fig)

# ================================================================
# 🧾 EXPORT DU RÉSUMÉ DES MÉTRIQUES
# ================================================================
with open(os.path.join(output_dir, "metrics_summary.txt"), "w") as f:
    f.write(f"Model: {model_name}\n")
    f.write(f"Epochs: {len(history_all['loss'])}\n\n")
    if history_all["loss"]:
        f.write(f"Final Training Loss: {history_all['loss'][-1]:.4f}\n")
    if history_all["val_loss"]:
        f.write(f"Final Validation Loss: {history_all['val_loss'][-1]:.4f}\n")
    if history_all["policy_acc"]:
        f.write(f"Final Policy Accuracy: {history_all['policy_acc'][-1]:.4f}\n")
    if history_all["val_policy_acc"]:
        f.write(f"Final Val Policy Accuracy: {history_all['val_policy_acc'][-1]:.4f}\n")
    if history_all["value_mae"]:
        f.write(f"Final Value MAE: {history_all['value_mae'][-1]:.4f}\n")
    if history_all["val_value_mae"]:
        f.write(f"Final Val Value MAE: {history_all['val_value_mae'][-1]:.4f}\n")

print(f"\n✅ Tous les fichiers et graphiques ont été enregistrés dans : {output_dir}")
